# for regression & classification

In [2]:
import pandas as pd
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import seaborn as sns 
from sklearn import datasets
from sklearn.model_selection import train_test_split ,StratifiedShuffleSplit , GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline



In [ ]:
iris = datasets.load_iris()
X = iris.data[:,:2]
y = iris.target
class_names = iris.target_names
class_names

## KNeighborsClassifier

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X.astype('float'))

X_train,X_test,y_train ,y_test = train_test_split(X_scaled,y,test_size=0.2,random_state=42)

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train,y_train)
knn.score(X_test,y_test)

In [ ]:
def plot_decision_boundaries(X,y,clf,resolution=0.2):
    #plot the decision surface
    x1_min,x1_max = X[:,0].min() - 1 ,X[:,0].max() + 1
    x2_min,x2_max = X[:,1].min() - 1 ,X[:,1].max() + 1
    
    xx1,xx2 = np.meshgrid(np.arange(x1_min,x1_max,resolution),
                         np.arange(x2_min,x2_max,resolution))

    Z = clf.predict(np.c_[xx1.ravel(),xx2.ravel()])
    Z = Z.reshape(xx1.shape)
    
    plt.contourf(xx1,xx2,Z,alpha=0.4)
    
    # Plot class samples
    plt.scatter(X[:,0],X[:,1],c=y,s = 20, edgecolors='k')
    
    plt.xlabel('Sepal length')
    plt.ylabel('Sepal width')
    
plot_decision_boundaries(X_scaled,y,knn)
plt.title('KNN (k=5)')
    

In [ ]:
neighbours,scores,scores_mean = [],[],[]

for n in range(1,75):
    neighbours.append(n)
    knn = KNeighborsClassifier(n_neighbors=n)
    score = cross_val_score(knn,X_scaled,y,cv=3)
    scores.append(score)
    scores_mean.append(score.mean())
    
display(scores,scores_mean)
    
plt.plot(neighbours,scores_mean)
plt.xlabel('k')
plt.ylabel('Accuracy')
plt.title('KNN')
plt.show()

#### Ex KNeighborsClassifier

In [ ]:
#Ex KNeighborsClassifier
mnist_path = 'mnist-original.mat'

mnist = loadmat(mnist_path)
X = mnist['data'].T
y = mnist['label'][0]

#shuffle
all_samples = 70000
shuffle_index = np.random.permutation(all_samples)
X_shuffle,y_shuffle = X[shuffle_index],y[shuffle_index]

# split to train & test 
train_size = 3000
X_train,y_train = X_shuffle[:train_size],y_shuffle[:train_size]

test_size = train_size + 3000
X_test,y_test = X_shuffle[train_size:test_size],y_shuffle[train_size:test_size]

display(X_train.shape,X_test.shape)




In [ ]:
clf = Pipeline([('cls',KNeighborsClassifier())])

param_grid = {
    'cls__weights': ['uniform','distance'],
    'cls__n_neighbors': [3,4,5,6,7,8]
}

grid_search = GridSearchCV(clf,param_grid,cv=3)
grid_search.fit(X_train,y_train)

print(grid_search.best_params_)

results = grid_search.cv_results_
for mean_score,params in zip(results['mean_test_score'],results['params']):
    print(mean_score,params)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3,weights='distance') 
score = cross_val_score(knn,X_test,y_test,cv=3)
display(score)

In [17]:
#grid search with StratifiedShuffleSplit
mnist_path = 'mnist-original.mat'

mnist = loadmat(mnist_path)
X = mnist['data'].T
y = mnist['label'][0]

sss = StratifiedShuffleSplit(n_splits=10, test_size=0.05,train_size=0.06, random_state=0)
train_index, test_index = next(sss.split(X=X, y=y))
     

X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]
display(X_train.shape,X_test.shape)

#clf = Pipeline([('cls',KNeighborsClassifier(algorithm='ball_tree'))])
clf = Pipeline([('cls',KNeighborsClassifier())])

param_grid = {
   'cls__weights': ['uniform','distance'],
    'cls__n_neighbors': np.arange(1, 12, 2),
    'cls__algorithm':['ball_tree','brute'],
    'cls__p':[1,2]
}

grid_search = GridSearchCV(clf,param_grid,cv=3)
grid_search.fit(X_train,y_train)

print(grid_search.best_params_)

results = grid_search.cv_results_
for mean_score,params in zip(results['mean_test_score'],results['params']):
    print(mean_score,params)

(4200, 784)

(3500, 784)

{'cls__algorithm': 'ball_tree', 'cls__n_neighbors': 3, 'cls__p': 2, 'cls__weights': 'distance'}
0.9019047619047619 {'cls__algorithm': 'ball_tree', 'cls__n_neighbors': 1, 'cls__p': 1, 'cls__weights': 'uniform'}
0.9019047619047619 {'cls__algorithm': 'ball_tree', 'cls__n_neighbors': 1, 'cls__p': 1, 'cls__weights': 'distance'}
0.909047619047619 {'cls__algorithm': 'ball_tree', 'cls__n_neighbors': 1, 'cls__p': 2, 'cls__weights': 'uniform'}
0.909047619047619 {'cls__algorithm': 'ball_tree', 'cls__n_neighbors': 1, 'cls__p': 2, 'cls__weights': 'distance'}
0.9016666666666666 {'cls__algorithm': 'ball_tree', 'cls__n_neighbors': 3, 'cls__p': 1, 'cls__weights': 'uniform'}
0.9069047619047619 {'cls__algorithm': 'ball_tree', 'cls__n_neighbors': 3, 'cls__p': 1, 'cls__weights': 'distance'}
0.9142857142857143 {'cls__algorithm': 'ball_tree', 'cls__n_neighbors': 3, 'cls__p': 2, 'cls__weights': 'uniform'}
0.9192857142857143 {'cls__algorithm': 'ball_tree', 'cls__n_neighbors': 3, 'cls__p': 2, 'cls__weights': 'd

In [16]:
knn = KNeighborsClassifier(n_neighbors=3,weights='distance',p=2) 
score = cross_val_score(knn,X_test,y_test,cv=3)
display(score)

array([0.89046942, 0.91428571, 0.88952654])